# Pre Requisits

### Loading Libraries and Dataset

In [ ]:
# Importing the necessary libraries

import os
import sys
import numpy as np
import pandas as pd
from datetime import datetime


import sklearn.metrics as metrics
sys.path.append(os.path.abspath(".."))
from utils.functions import ModelSolutions

from sklearn.utils import resample

from xgboost import XGBClassifier, plot_importance
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder

pd.set_option('display.max_columns', None)
import warnings

warnings.filterwarnings('ignore')

model_solution = ModelSolutions()

# Load dataset
# df = model_solution.load_data(folders=['data','raw'],file_name='E-Commerce_data.csv')

In [ ]:
# Selecting only the required columns

data = df[[
        'id',
        'category',
        'product',
        'purchase_date',
        'purchase_amount',
        'returned',
        'Preferred_Shopping_Channels'
    ]].sort_values(by='id').reset_index().drop('index',axis=1)


data['purchase_date'] = pd.to_datetime(data['purchase_date'])

data = data.sort_values(['id','purchase_date'])


data.dropna(inplace=True)
data.head()

,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels
0,CUS1000021605,Electronics,Smart Thermostat,2022-07-21 03:22:54,100.00,no,In-store
5,CUS1000092910,Tools & Hardware,Wire Cutters,2022-02-11 23:24:48,56.25,no,Website
10,CUS1000092910,Office Supplies & Stationery,Rubber Bands,2022-03-12 03:58:02,2.50,exchange,Website
4,CUS1000092910,Toys & Games,LEGO Sets,2022-05-15 08:29:27,18.75,no,Website
8,CUS1000092910,Food & Beverages,Kombucha,2022-05-23 16:06:13,3.75,no,Website


In [3]:
data_sample = data.copy()
data_sample.shape

(3349140, 7)

## Testing 

In [32]:



def record_entries(
        
        product_name:str=None,
        category_name:str=None,
        purchase_amount:int | float=None,
        preffered_shopping_channel:str=None,
        # data_file:Any=None
        
        ):
    

    current_date = pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    id = str(np.random.randint(10000,1000000))
    id = 'CUS'+id
    returned = 'no'
    new_row = {
        "id": id,
        "category": category_name,
        "product": product_name,
        "purchase_date":             current_date,
        "purchase_amount":           purchase_amount,
        "returned":          returned,
        "Preferred_Shopping_Channels": preffered_shopping_channel
    }
    
    new_row = pd.DataFrame([new_row])
    # data_file = pd.concat([data_file,new_row],ignore_index=True)
    
    print(id)
    return new_row

data12 = record_entries(
    product_name='Wire Cutters',
    category_name='Tools & Hardware',
    purchase_amount=100,
    preffered_shopping_channel='Website',
    # data_file=data_sample
    )
data12

CUS180732


,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels
0,CUS180732,Tools & Hardware,Wire Cutters,2025-05-18 13:14:30,100,no,Website


In [5]:
col_1 = processed_data.columns.to_list()
col_2 = balanced_data.columns.to_list()

In [6]:
pro_arr = np.array(processed_data)
bal_arr = np.array(balanced_data)

In [8]:
col_1

['id',
 'category',
 'product',
 'purchase_date',
 'purchase_amount',
 'returned',
 'Preferred_Shopping_Channels',
 'purchases',
 'return_rate',
 'exchange_rate',
 'kept_rate',
 'segment',
 'days_since_last',
 'amt_bin',
 'cat_refund_rate',
 'cat_exchange_rate',
 'cat_kept_rate',
 'channel_return_rate',
 'channel_exchange_rate',
 'channel_keep_rate']

In [46]:
category_name='Tools & Hardware'
preffered_shopping_channel='Website'

# pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,17]
pro_arr[pro_arr[:,1]==category_name][:,14:17]

array([[0.0247759739891923, 0.3098129623545672, 0.6654110636562405],
       [0.0247759739891923, 0.3098129623545672, 0.6654110636562405],
       [0.0247759739891923, 0.3098129623545672, 0.6654110636562405],
       ...,
       [0.0247759739891923, 0.3098129623545672, 0.6654110636562405],
       [0.0247759739891923, 0.3098129623545672, 0.6654110636562405],
       [0.0247759739891923, 0.3098129623545672, 0.6654110636562405]],
      shape=(176542, 3), dtype=object)

In [51]:
pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,17][0]

0.0216437509484371

In [41]:
processed_data[processed_data['Preferred_Shopping_Channels']==preffered_shopping_channel].head(5)

,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
1,CUS1000092910,Tools & Hardware,Wire Cutters,2022-02-11 23:24:48,56.25,no,Website,12,0.0,50.0,50.0,loyal_buyer,-1.0,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874
2,CUS1000092910,Office Supplies & Stationery,Rubber Bands,2022-03-12 03:58:02,2.50,exchange,Website,12,0.0,50.0,50.0,loyal_buyer,28.0,very_low,0.023359,0.302716,0.673926,0.021644,0.309616,0.66874
3,CUS1000092910,Toys & Games,LEGO Sets,2022-05-15 08:29:27,18.75,no,Website,12,0.0,50.0,50.0,loyal_buyer,64.0,very_low,0.025357,0.310438,0.664206,0.021644,0.309616,0.66874
4,CUS1000092910,Food & Beverages,Kombucha,2022-05-23 16:06:13,3.75,no,Website,12,0.0,50.0,50.0,loyal_buyer,8.0,very_low,0.024019,0.300599,0.675382,0.021644,0.309616,0.66874
5,CUS1000092910,Tools & Hardware,Tape Measure,2022-06-02 06:53:11,2.50,exchange,Website,12,0.0,50.0,50.0,loyal_buyer,9.0,very_low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


In [ ]:
segment_labels = ['one_time_buyer','infrequent_buyer','frequent_buyer','loyal_buyer']
segment_bins = [0, 2, 5, 11, float('inf')]

amt_bins = [0,50,200,500,1000,float('inf')]
amt_labels = ['very_low','low','mid','high','very_high']



current_date = pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
id = 'CUS'+str(np.random.randint(10000,1000000))
returned = 'no'
category_name='Tools & Hardware'
product_name='Wire Cutters'
purchase_amount=100
preffered_shopping_channel='Website'


new_arr = np.array([id,category_name,product_name,current_date,purchase_amount,returned,preffered_shopping_channel])

purchases = len(new_arr[new_arr[0]==id])

kept = len(new_arr[new_arr[5]=='no'])
exchange = len(new_arr[new_arr[5]=='exchange'])
refund = len(new_arr[new_arr[5]=='refund'])

return_rate = (refund / purchases)*100
exchange_rate = (exchange / purchases)*100
kept_rate = (kept/purchases)*100

segments = pd.cut([purchases],labels=segment_labels, bins=segment_bins,right=False)[0]


segments = pd.cut([purchases],labels=segment_labels, bins=segment_bins,right=False)[0]
# days_since_last_purchase = new_row['purchase_date'].sort_values().diff().dt.days.fillna(-1)[0]

# cat_refund = float(processed_data[processed_data['category']==category_name]['cat_refund_rate'].iloc[0])
# cat_exchnage = float(processed_data[processed_data['category']==category_name]['cat_exchange_rate'].iloc[0])
# cat_kept = float(processed_data[processed_data['category']==category_name]['cat_kept_rate'].iloc[0])
cat_refund = pro_arr[pro_arr[:,1]==category_name][:,14][0]
cat_exchnage = pro_arr[pro_arr[:,1]==category_name][:,15][0]
cat_kept = pro_arr[pro_arr[:,1]==category_name][:,16][0]


# chan_refund = float(processed_data[processed_data['Preferred_Shopping_Channels']==preffered_shopping_channel]['channel_return_rate'].iloc[0])
# chan_exchange = float(processed_data[processed_data['Preferred_Shopping_Channels']==preffered_shopping_channel]['channel_exchange_rate'].iloc[0])
# chan_kept = float(processed_data[processed_data['Preferred_Shopping_Channels']==preffered_shopping_channel]['channel_keep_rate'].iloc[0])
chan_refund = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,17][0]
chan_exchange = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,18][0]
chan_kept = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,19][0]

amt_bin = pd.cut([new_arr[4]],labels=amt_labels,bins=amt_bins)[0]

days_since_last_purchase = 0

#'returns'
#'exchanges'
#'kept'
#'return_rate'
#'exchange_rate'
#'kept_rate'
#'segment':
#'days_since_last'
#'amt_bin'
#'cat_refund_rate'
#'cat_exchange_rate'
#'cat_kept_rate'
#'channel_return_rate'
#'channel_exchange_rate'
#'channel_keep_rate'

features = np.array([
    purchases,
    # int(refund),
    # int(exchange),
    # int(kept),
    float(return_rate),
    float(exchange_rate),
    float(kept_rate),
    segments,
    float(days_since_last_purchase),
    amt_bin,
    cat_refund,
    cat_exchnage,
    cat_kept,
    chan_refund,
    chan_exchange,
    chan_kept
    ],dtype=object)
completed_data = np.concatenate((new_arr,features))

completed_data

array(['CUS886687', 'Tools & Hardware', 'Wire Cutters',
       Timestamp('2025-05-18 20:39:40'), 100, 'no', 'Website', 1, 0.0,
       0.0, 100.0, 'one_time_buyer', 0.0, 'low', 0.0247759739891923,
       0.3098129623545672, 0.6654110636562405, 0.0216437509484371,
       0.3096164049441682, 0.6687398441073946], dtype=object)

In [61]:
def entry_processing(
        
        product_name:str=None,
        category_name:str=None,
        purchase_amount:int | float=None,
        preffered_shopping_channel:str=None,
        processed_df:pd.DataFrame=None,
        encoded_data:pd.DataFrame=None,
        le:LabelEncoder=None,
        scaler:StandardScaler=None):
    col_1 = processed_df.columns.to_list()
    col_2 = encoded_data.columns.to_list()
    pro_arr = np.array(processed_df)
    bal_arr = np.array(encoded_data)
    segment_labels = ['one_time_buyer','infrequent_buyer','frequent_buyer','loyal_buyer']
    segment_bins = [0, 2, 5, 11, float('inf')]
    amt_bins = [0,50,200,500,1000,float('inf')]
    amt_labels = ['very_low','low','mid','high','very_high']
    
    current_date = pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    id = 'CUS'+str(np.random.randint(10000,1000000))
    returned = 'no'
    new_arr = np.array([id,category_name,product_name,current_date,purchase_amount,returned,preffered_shopping_channel])

    purchases = len(new_arr[new_arr[0]==id])
    
    kept = len(new_arr[new_arr[5]=='no'])
    exchange = len(new_arr[new_arr[5]=='exchange'])
    refund = len(new_arr[new_arr[5]=='refund'])
    
    return_rate = (refund / purchases)*100
    exchange_rate = (exchange / purchases)*100
    kept_rate = (kept/purchases)*100
    
    segments = pd.cut([purchases],labels=segment_labels, bins=segment_bins,right=False)[0]

    cat_refund = pro_arr[pro_arr[:,1]==category_name][:,14][0]
    cat_exchnage = pro_arr[pro_arr[:,1]==category_name][:,15][0]
    cat_kept = pro_arr[pro_arr[:,1]==category_name][:,16][0]

    chan_refund = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,17][0]
    chan_exchange = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,18][0]
    chan_kept = pro_arr[pro_arr[:,6]==preffered_shopping_channel][:,19][0]

    amt_bin = pd.cut([new_arr[4]],labels=amt_labels,bins=amt_bins)[0]

    days_since_last_purchase = 0
    features = np.array([
    purchases,
    float(return_rate),
    float(exchange_rate),
    float(kept_rate),
    segments,
    float(days_since_last_purchase),
    amt_bin,
    cat_refund,
    cat_exchnage,
    cat_kept,
    chan_refund,
    chan_exchange,
    chan_kept
    ],dtype=object)
    completed_data = np.concatenate((new_arr,features))
    columns = processed_df.columns.to_list()
    completed_data = pd.DataFrame([completed_data],columns=columns)
    encoded_complete_data = completed_data.drop(columns=['id','purchase_date'])
    encoded_complete_data['purchase_amount'] = scaler.transform(encoded_complete_data[['purchase_amount']])
    encoded_complete_data['returned'] = le.transform(encoded_complete_data['returned'])
    new_row = pd.get_dummies(encoded_complete_data)
    col_list = encoded_data.columns.to_list()
    test_data = pd.DataFrame(columns=col_list)
    sample_test = pd.concat([test_data,new_row])
    sample_test.fillna(False,inplace=True)
    return sample_test
    


In [62]:
balanced_data.head(1)

,purchase_amount,returned,purchases,return_rate,exchange_rate,kept_rate,days_since_last,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate,category_Automotive Parts & Accessories,category_Baby & Maternity Products,category_Beauty & Personal Care,category_Books & Literature,category_Clothing & Apparel,category_Electronics,category_Food & Beverages,category_Furniture & Home Decor,category_Health & Wellness Products,category_Home & Kitchen Appliances,category_Jewelry & Watches,category_Musical Instruments,category_Office Supplies & Stationery,category_Pet Supplies,category_Sports & Outdoor Equipment,category_Tools & Hardware,category_Toys & Games,product_3D Printer,product_Accent Chair,product_Accent Tables,product_Accordion,product_Acne Treatment,product_Action Camera,product_Action Figures,product_Adjustable Wrench,product_Aftershave,product_Air Fryer,product_Air Purifier,product_Aloe Vera Gel,product_Ankle Bracelet,product_Anklet,product_Anti-aging Cream,product_Arduino,product_Art Books,product_Autobiographies,product_Award-winning Books,product_Baby Bath Tub,product_Baby Books,product_Baby Bottles,product_Baby Clothes,product_Baby Food Maker,product_Baby High Chair,product_Baby Monitor,product_Baby Shoes,product_Baby Sunscreen,product_Baby Thermometer,product_Baby Toys,product_Baby Wash,product_Badge Holder,product_Bagpipes,product_Bangle,product_Basketball,product_Bass Drum,product_Bass Guitar,product_Bathroom Vanity,product_Bean Bag Chair,product_Beer,product_Belly Button Ring,product_Belt,product_Bestsellers,product_Bike Rack,product_Bird Cage,product_Birthstone Jewelry,product_Blankets,product_Blender,product_Blouse,product_Board Games,product_Boat Cover,product_Body Lotion,product_Body Oil,product_Body Scrub,product_Bodysuit,product_Bookshelf,product_Boots,product_Bracelet,product_Brake Pads,product_Bread Maker,product_Breadboard,product_Brooch Pin,product_Building Blocks,product_Cable Organizer,product_Camera,product_Camping Tent,product_Candles,product_Candy,product_Canned Goods,product_Canoe,product_Car Drying Towel,product_Car GPS Navigator,product_Car Mats,product_Car Wash Mitt,product_Car Wax,product_Card Games,product_Cardigan,product_Cargo Carrier,product_Cat Collar,product_Cat Litter,product_Cat Shampoo,product_Chain Bracelet,product_Chainsaw,product_Chandeliers,product_Changing Pad,product_Charm,product_Chips,product_Choker,product_Choker Necklace,product_Circular Saw,product_Clarinet,product_Coat,product_Coat Rack,product_Coconut Water,product_Coffee,product_Coffee Maker,product_Coffee Table,product_Comic Books,product_Computer Monitor,product_Conditioner,product_Conga Drums,product_Console Table,product_Convection Oven,product_Countertop Blender,product_Cowbell,product_Cufflinks,product_Curl Cream,product_Curtains,product_Cycling Helmet,product_Dehumidifier,product_Desk,product_Desk Calendar,product_Desk Lamp,product_Desk Organizer,product_Detox Foot Pads,product_Diaper Bag,product_Diaper Pail,product_Diaper Rash Cream,product_Didgeridoo,product_Digestive Enzymes,product_Dining Table,product_Dog Backpack,product_Dog Boots,product_Dog Collar,product_Dog Shampoo,product_Dog Sweater,product_Dress,product_Dresser,product_Drill,product_Dry Shampoo,product_Dumbbells,product_Dust Mask,product_E-book Reader,product_Earrings,product_Educational Board Games,product_Educational Books,product_Educational Toys,product_Electric Can Opener,product_Electric Guitar,product_Electric Massager,product_Electric Smoker,product_Electric Toothbrush,product_End Table,product_Energy Bars,product_Energy Drinks,product_Envelopes,product_Essays,product_Essential Oil Diffuser,product_Ethernet Cable,product_Exercise Bike,product_External Hard Drive,product_Eye Cream,product_Face Cleanser,product_Facial Mask,product_Facial Moisturizer,product_Facial Oil,product_Fantasy Novels,product_Fiber Supplements,product_Fiction Novels,product_Filing Cabinet,product_Fishing Kayak,product_Fitness

In [59]:
entry_processing(
    product_name='Wire Cutters',
    category_name='Tools & Hardware',
    purchase_amount=10,
    processed_df=processed_data,
    encoded_data=balanced_data,
    preffered_shopping_channel='Website')

,purchase_amount,returned,purchases,return_rate,exchange_rate,kept_rate,days_since_last,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate,category_Automotive Parts & Accessories,category_Baby & Maternity Products,category_Beauty & Personal Care,category_Books & Literature,category_Clothing & Apparel,category_Electronics,category_Food & Beverages,category_Furniture & Home Decor,category_Health & Wellness Products,category_Home & Kitchen Appliances,category_Jewelry & Watches,category_Musical Instruments,category_Office Supplies & Stationery,category_Pet Supplies,category_Sports & Outdoor Equipment,category_Tools & Hardware,category_Toys & Games,product_3D Printer,product_Accent Chair,product_Accent Tables,product_Accordion,product_Acne Treatment,product_Action Camera,product_Action Figures,product_Adjustable Wrench,product_Aftershave,product_Air Fryer,product_Air Purifier,product_Aloe Vera Gel,product_Ankle Bracelet,product_Anklet,product_Anti-aging Cream,product_Arduino,product_Art Books,product_Autobiographies,product_Award-winning Books,product_Baby Bath Tub,product_Baby Books,product_Baby Bottles,product_Baby Clothes,product_Baby Food Maker,product_Baby High Chair,product_Baby Monitor,product_Baby Shoes,product_Baby Sunscreen,product_Baby Thermometer,product_Baby Toys,product_Baby Wash,product_Badge Holder,product_Bagpipes,product_Bangle,product_Basketball,product_Bass Drum,product_Bass Guitar,product_Bathroom Vanity,product_Bean Bag Chair,product_Beer,product_Belly Button Ring,product_Belt,product_Bestsellers,product_Bike Rack,product_Bird Cage,product_Birthstone Jewelry,product_Blankets,product_Blender,product_Blouse,product_Board Games,product_Boat Cover,product_Body Lotion,product_Body Oil,product_Body Scrub,product_Bodysuit,product_Bookshelf,product_Boots,product_Bracelet,product_Brake Pads,product_Bread Maker,product_Breadboard,product_Brooch Pin,product_Building Blocks,product_Cable Organizer,product_Camera,product_Camping Tent,product_Candles,product_Candy,product_Canned Goods,product_Canoe,product_Car Drying Towel,product_Car GPS Navigator,product_Car Mats,product_Car Wash Mitt,product_Car Wax,product_Card Games,product_Cardigan,product_Cargo Carrier,product_Cat Collar,product_Cat Litter,product_Cat Shampoo,product_Chain Bracelet,product_Chainsaw,product_Chandeliers,product_Changing Pad,product_Charm,product_Chips,product_Choker,product_Choker Necklace,product_Circular Saw,product_Clarinet,product_Coat,product_Coat Rack,product_Coconut Water,product_Coffee,product_Coffee Maker,product_Coffee Table,product_Comic Books,product_Computer Monitor,product_Conditioner,product_Conga Drums,product_Console Table,product_Convection Oven,product_Countertop Blender,product_Cowbell,product_Cufflinks,product_Curl Cream,product_Curtains,product_Cycling Helmet,product_Dehumidifier,product_Desk,product_Desk Calendar,product_Desk Lamp,product_Desk Organizer,product_Detox Foot Pads,product_Diaper Bag,product_Diaper Pail,product_Diaper Rash Cream,product_Didgeridoo,product_Digestive Enzymes,product_Dining Table,product_Dog Backpack,product_Dog Boots,product_Dog Collar,product_Dog Shampoo,product_Dog Sweater,product_Dress,product_Dresser,product_Drill,product_Dry Shampoo,product_Dumbbells,product_Dust Mask,product_E-book Reader,product_Earrings,product_Educational Board Games,product_Educational Books,product_Educational Toys,product_Electric Can Opener,product_Electric Guitar,product_Electric Massager,product_Electric Smoker,product_Electric Toothbrush,product_End Table,product_Energy Bars,product_Energy Drinks,product_Envelopes,product_Essays,product_Essential Oil Diffuser,product_Ethernet Cable,product_Exercise Bike,product_External Hard Drive,product_Eye Cream,product_Face Cleanser,product_Facial Mask,product_Facial Moisturizer,product_Facial Oil,product_Fantasy Novels,product_Fiber Supplements,product_Fiction Novels,product_Filing Cabinet,product_Fishing Kayak,product_Fitness

In [164]:
columns = processed_data.columns.to_list()

completed_data = pd.DataFrame([completed_data],columns=columns)

completed_data

,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,CUS886687,Tools & Hardware,Wire Cutters,2025-05-18 20:39:40,100,no,Website,1,0.0,0.0,100.0,one_time_buyer,0.0,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


In [165]:
completed_data['purchase_amount'] = scaler.transform(completed_data[['purchase_amount']])
completed_data

,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,CUS886687,Tools & Hardware,Wire Cutters,2025-05-18 20:39:40,-0.039818,no,Website,1,0.0,0.0,100.0,one_time_buyer,0.0,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


In [172]:
encoded_complete_data = completed_data.drop(columns=['id','purchase_date'])
encoded_complete_data

,category,product,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,Tools & Hardware,Wire Cutters,-0.039818,no,Website,1,0.0,0.0,100.0,one_time_buyer,0.0,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


In [173]:
encoded_complete_data['returned'] = le.transform(encoded_complete_data['returned'])
encoded_complete_data

,category,product,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,Tools & Hardware,Wire Cutters,-0.039818,1,Website,1,0.0,0.0,100.0,one_time_buyer,0.0,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


In [177]:
new_row = pd.get_dummies(encoded_complete_data)

In [169]:
balanced_data.head(1)

,purchase_amount,returned,purchases,return_rate,exchange_rate,kept_rate,days_since_last,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate,category_Automotive Parts & Accessories,category_Baby & Maternity Products,category_Beauty & Personal Care,category_Books & Literature,category_Clothing & Apparel,category_Electronics,category_Food & Beverages,category_Furniture & Home Decor,category_Health & Wellness Products,category_Home & Kitchen Appliances,category_Jewelry & Watches,category_Musical Instruments,category_Office Supplies & Stationery,category_Pet Supplies,category_Sports & Outdoor Equipment,category_Tools & Hardware,category_Toys & Games,product_3D Printer,product_Accent Chair,product_Accent Tables,product_Accordion,product_Acne Treatment,product_Action Camera,product_Action Figures,product_Adjustable Wrench,product_Aftershave,product_Air Fryer,product_Air Purifier,product_Aloe Vera Gel,product_Ankle Bracelet,product_Anklet,product_Anti-aging Cream,product_Arduino,product_Art Books,product_Autobiographies,product_Award-winning Books,product_Baby Bath Tub,product_Baby Books,product_Baby Bottles,product_Baby Clothes,product_Baby Food Maker,product_Baby High Chair,product_Baby Monitor,product_Baby Shoes,product_Baby Sunscreen,product_Baby Thermometer,product_Baby Toys,product_Baby Wash,product_Badge Holder,product_Bagpipes,product_Bangle,product_Basketball,product_Bass Drum,product_Bass Guitar,product_Bathroom Vanity,product_Bean Bag Chair,product_Beer,product_Belly Button Ring,product_Belt,product_Bestsellers,product_Bike Rack,product_Bird Cage,product_Birthstone Jewelry,product_Blankets,product_Blender,product_Blouse,product_Board Games,product_Boat Cover,product_Body Lotion,product_Body Oil,product_Body Scrub,product_Bodysuit,product_Bookshelf,product_Boots,product_Bracelet,product_Brake Pads,product_Bread Maker,product_Breadboard,product_Brooch Pin,product_Building Blocks,product_Cable Organizer,product_Camera,product_Camping Tent,product_Candles,product_Candy,product_Canned Goods,product_Canoe,product_Car Drying Towel,product_Car GPS Navigator,product_Car Mats,product_Car Wash Mitt,product_Car Wax,product_Card Games,product_Cardigan,product_Cargo Carrier,product_Cat Collar,product_Cat Litter,product_Cat Shampoo,product_Chain Bracelet,product_Chainsaw,product_Chandeliers,product_Changing Pad,product_Charm,product_Chips,product_Choker,product_Choker Necklace,product_Circular Saw,product_Clarinet,product_Coat,product_Coat Rack,product_Coconut Water,product_Coffee,product_Coffee Maker,product_Coffee Table,product_Comic Books,product_Computer Monitor,product_Conditioner,product_Conga Drums,product_Console Table,product_Convection Oven,product_Countertop Blender,product_Cowbell,product_Cufflinks,product_Curl Cream,product_Curtains,product_Cycling Helmet,product_Dehumidifier,product_Desk,product_Desk Calendar,product_Desk Lamp,product_Desk Organizer,product_Detox Foot Pads,product_Diaper Bag,product_Diaper Pail,product_Diaper Rash Cream,product_Didgeridoo,product_Digestive Enzymes,product_Dining Table,product_Dog Backpack,product_Dog Boots,product_Dog Collar,product_Dog Shampoo,product_Dog Sweater,product_Dress,product_Dresser,product_Drill,product_Dry Shampoo,product_Dumbbells,product_Dust Mask,product_E-book Reader,product_Earrings,product_Educational Board Games,product_Educational Books,product_Educational Toys,product_Electric Can Opener,product_Electric Guitar,product_Electric Massager,product_Electric Smoker,product_Electric Toothbrush,product_End Table,product_Energy Bars,product_Energy Drinks,product_Envelopes,product_Essays,product_Essential Oil Diffuser,product_Ethernet Cable,product_Exercise Bike,product_External Hard Drive,product_Eye Cream,product_Face Cleanser,product_Facial Mask,product_Facial Moisturizer,product_Facial Oil,product_Fantasy Novels,product_Fiber Supplements,product_Fiction Novels,product_Filing Cabinet,product_Fishing Kayak,product_Fitness

In [179]:
col_list = balanced_data.columns.to_list()

test_data = pd.DataFrame(columns=col_list)

In [185]:
sample_test = pd.concat([test_data,new_row])

C:\Users\tohid\AppData\Local\Temp\ipykernel_7340\216414472.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sample_test = pd.concat([test_data,new_row])


In [ ]:
sample_test.fillna(False,inplace=True)

0    True
Name: category_Tools & Hardware, dtype: object

In [134]:
current_date = pd.to_datetime(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
id = 'CUS'+str(np.random.randint(10000,1000000))
returned = 'no'
category_name='Tools & Hardware'
product_name='Wire Cutters'
purchase_amount=100
preffered_shopping_channel='Website'

new_row = pd.DataFrame([{
        "id": id,
        "category": category_name,
        "product": product_name,
        "purchase_date":             current_date,
        "purchase_amount":           purchase_amount,
        "returned":          returned,
        "Preferred_Shopping_Channels": preffered_shopping_channel}])


segment_labels = ['one_time_buyer','infrequent_buyer','frequent_buyer','loyal_buyer']
segment_bins = [0, 2, 5, 11, float('inf')]

amt_bins = [0,50,200,500,1000,float('inf')]
amt_labels = ['very_low','low','mid','high','very_high']


purchases = new_row.shape[0]
kept = (new_row['returned']=='no').sum()
exchange = (new_row['returned']=='exchange').sum()
refund = (new_row['returned']=='refund').sum()
return_rate = (refund / purchases)*100
exchange_rate = (exchange / purchases)*100
kept_rate = (kept/purchases)*100

segments = pd.cut([purchases],labels=segment_labels, bins=segment_bins,right=False)[0]
days_since_last_purchase = new_row['purchase_date'].sort_values().diff().dt.days.fillna(-1)[0]

cat_refund = float(processed_data[processed_data['category']==category_name]['cat_refund_rate'].iloc[0])
cat_exchnage = float(processed_data[processed_data['category']==category_name]['cat_exchange_rate'].iloc[0])
cat_kept = float(processed_data[processed_data['category']==category_name]['cat_kept_rate'].iloc[0])


chan_refund = float(processed_data[processed_data['Preferred_Shopping_Channels']==new_row['Preferred_Shopping_Channels'].iloc[0]]['channel_return_rate'].iloc[0])
chan_exchange = float(processed_data[processed_data['Preferred_Shopping_Channels']==new_row['Preferred_Shopping_Channels'].iloc[0]]['channel_exchange_rate'].iloc[0])
chan_kept = float(processed_data[processed_data['Preferred_Shopping_Channels']==new_row['Preferred_Shopping_Channels'].iloc[0]]['channel_keep_rate'].iloc[0])

amt_bin = pd.cut(new_row['purchase_amount'],labels=amt_labels,bins=amt_bins)[0]
id = new_row['id'][0]

result = pd.DataFrame([{
    'id': id,
    'purchases': purchases,
    # 'returns': int(refund),
    # 'exchanges': int(exchange),
    # 'kept': int(kept),
    'return_rate': float(return_rate),
    'exchange_rate': float(exchange_rate),
    'kept_rate': float(kept_rate),
    'segment': segments,
    'days_since_last': int(days_since_last_purchase),
    'amt_bin': amt_bin,
    'cat_refund_rate':cat_refund,
    'cat_exchange_rate':cat_exchnage,
    'cat_kept_rate':cat_kept,
    'channel_return_rate':chan_refund,
    'channel_exchange_rate':chan_exchange,
    'channel_keep_rate':chan_kept
}])

completed_data = pd.merge(new_row,result,on='id',how='left')

completed_data


,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels,purchases,returns,exchanges,kept,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,CUS157764,Tools & Hardware,Wire Cutters,2025-05-18 19:17:45,100,no,Website,1,0,0,1,0.0,0.0,100.0,one_time_buyer,-1,low,0.024776,0.309813,0.665411,0.021644,0.309616,0.66874


0.0216437509484371

## Testing End

## Main Data Continue

In [4]:
cus_data = data.groupby(['id']).agg(
    purchases=('id','count'),
    returns=('returned',lambda x:  (x=='refund').sum()),
    exchanges=('returned',lambda x: (x=='exchange').sum()),
    kept = ('returned',lambda x: (x=='no').sum())
    ).reset_index()

cus_data.head()

,id,purchases,returns,exchanges,kept
0,CUS1000021605,1,0,0,1
1,CUS1000092910,12,0,6,6
2,CUS1000126344,7,1,2,4
3,CUS1000136246,15,0,4,11
4,CUS1000143478,3,1,0,2
...,...,...,...,...,...
359995,CUS9999919407,4,1,2,1
359996,CUS9999921029,7,0,0,7
359997,CUS9999959196,1,1,0,0
359998,CUS9999983244,1,1,0,0


In [5]:
data_sample = cus_data.copy()

labels = ['one_time_buyer','infrequent_buyer','frequent_buyer','loyal_buyer']
# bins = [0,1,4,10,float('inf')]
bins = [0, 2, 5, 11, float('inf')]

data_sample['segment'] = pd.cut(data_sample['purchases'],bins=bins,labels=labels,right=False)

data_sample['return_rate'] = round((data_sample['returns'] / data_sample['purchases'])*100,2)
data_sample['exchange_rate'] = round((data_sample['exchanges'] / data_sample['purchases'])*100,2)
data_sample['kept_rate'] = round((data_sample['kept'] / data_sample['purchases'])*100,2)

data_sample

,id,purchases,returns,exchanges,kept,segment,return_rate,exchange_rate,kept_rate
0,CUS1000021605,1,0,0,1,one_time_buyer,0.00,0.00,100.00
1,CUS1000092910,12,0,6,6,loyal_buyer,0.00,50.00,50.00
2,CUS1000126344,7,1,2,4,frequent_buyer,14.29,28.57,57.14
3,CUS1000136246,15,0,4,11,loyal_buyer,0.00,26.67,73.33
4,CUS1000143478,3,1,0,2,infrequent_buyer,33.33,0.00,66.67
...,...,...,...,...,...,...,...,...,...
359995,CUS9999919407,4,1,2,1,infrequent_buyer,25.00,50.00,25.00
359996,CUS9999921029,7,0,0,7,frequent_buyer,0.00,0.00,100.00
359997,CUS9999959196,1,1,0,0,one_time_buyer,100.00,0.00,0.00
359998,CUS9999983244,1,1,0,0,one_time_buyer,100.00,0.00,0.00


In [ ]:
data_22 = data.copy()


data_22 = data_22.merge(
    data_sample[
        ['id','purchases','return_rate','exchange_rate','kept_rate','segment']
    ],
    on='id',
    how='left'
)

data_22['days_since_last'] = data_22.groupby('id')['purchase_date'].diff().dt.days.fillna(-1)

cat_ref = data_22.groupby('category')['returned'].apply(lambda x: (x=='refund').mean())
cat_exch = data_22.groupby('category')['returned'].apply(lambda x: (x=='exchange').mean())
cat_keep = data_22.groupby('category')['returned'].apply(lambda x: (x=='no').mean())

chan_ref = data_22.groupby('Preferred_Shopping_Channels')['returned'].apply(lambda x: (x=='refund').mean())
chan_exch = data_22.groupby('Preferred_Shopping_Channels')['returned'].apply(lambda x: (x=='exchange').mean())
chan_keep = data_22.groupby('Preferred_Shopping_Channels')['returned'].apply(lambda x: (x=='no').mean())

bins = [0,10,50,200,500,1000,float('inf')]

labels = ['very_low','low','mid','high','very_high','VIP']

data_22['amt_bin'] = pd.cut(data_22['purchase_amount'],bins=bins,labels=labels)

data_22['cat_refund_rate'] = data_22['category'].map(cat_ref)
data_22['cat_exchange_rate'] = data_22['category'].map(cat_exch)
data_22['cat_kept_rate'] = data_22['category'].map(cat_keep)

data_22['channel_return_rate'] = data_22['Preferred_Shopping_Channels'].map(chan_ref)
data_22['channel_exchange_rate'] = data_22['Preferred_Shopping_Channels'].map(chan_exch)
data_22['channel_keep_rate'] = data_22['Preferred_Shopping_Channels'].map(chan_keep)

data_22

,id,category,product,purchase_date,purchase_amount,returned,Preferred_Shopping_Channels,purchases,return_rate,exchange_rate,kept_rate,segment,days_since_last,amt_bin,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate
0,CUS1000021605,Electronics,Smart Thermostat,2022-07-21 03:22:54,100.00,no,In-store,1,0.0,0.0,100.0,one_time_buyer,-1.0,mid,0.025947,0.311064,0.662989,0.023254,0.303718,0.673029
1,CUS1000092910,Tools & Hardware,Wire Cutters,2022-02-11 23:24:48,56.25,no,Website,12,0.0,50.0,50.0,loyal_buyer,-1.0,mid,0.024776,0.309813,0.665411,0.021644,0.309616,0.668740
2,CUS1000092910,Office Supplies & Stationery,Rubber Bands,2022-03-12 03:58:02,2.50,exchange,Website,12,0.0,50.0,50.0,loyal_buyer,28.0,very_low,0.023359,0.302716,0.673926,0.021644,0.309616,0.668740
3,CUS1000092910,Toys & Games,LEGO Sets,2022-05-15 08:29:27,18.75,no,Website,12,0.0,50.0,50.0,loyal_buyer,64.0,low,0.025357,0.310438,0.664206,0.021644,0.309616,0.668740
4,CUS1000092910,Food & Beverages,Kombucha,2022-05-23 16:06:13,3.75,no,Website,12,0.0,50.0,50.0,loyal_buyer,8.0,very_low,0.024019,0.300599,0.675382,0.021644,0.309616,0.668740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3349135,CUS9999989234,Tools & Hardware,Pipe Wrench,2022-08-25 19:41:05,56.25,no,Mobile App,8,12.5,12.5,75.0,frequent_buyer,3.0,mid,0.024776,0.309813,0.665411,0.029187,0.312867,0.657946
3349136,CUS9999989234,Home & Kitchen Appliances,Air Fryer,2023-03-12 06:52:05,150.00,no,Mobile App,8,12.5,12.5,75.0,frequent_buyer,198.0,mid,0.025628,0.312015,0.662357,0.029187,0.312867,0.657946
3349137,CUS9999989234,Food & Beverages,Frozen Meals,2023-05-04 13:47:26,37.50,no,Mobile App,8,12.5,12.5,75.0,frequent_buyer,53.0,low,0.024019,0.300599,0.675382,0.029187,0.312867,0.657946
3349138,CUS9999989234,Sports & Outdoor Equipment,Hiking Boots,2023-05-26 01:56:40,100.00,no,Mobile App,8,12.5,12.5,75.0,frequent_buyer,21.0,mid,0.025194,0.311054,0.663753,0.029187,0.312867,0.657946


# Data Preprocessing

In [ ]:
processed = model_solution.load_data(folders=['data','processed'],file_name='processed_data.csv')
grid_decision_tree = model_solution.load_model(folders=['models','grid_search_models'],file_name='grid_decisiontree_model.pkl')
grid_random_forest = model_solution.load_model(folders=['models','grid_search_models'],file_name='grid_forest.pkl')

In [76]:
# 1. Sampling to balance classes
kept_sample = resample(
    processed[processed['returned'] == 'no'], 
    replace=False, 
    n_samples=41000,
    random_state=50 
)
exchange_sample = resample(
    processed[processed['returned'] == 'exchange'], 
    replace=False, 
    n_samples=41000,
    random_state=50 
)
refund_sample = resample(
    processed[processed['returned'] == 'refund'], 
    replace=False, 
    n_samples=41000,
    random_state=50
)

# 2. Combine and shuffle
balanced_data = pd.concat([kept_sample, exchange_sample, refund_sample])
balanced_data = balanced_data.sample(frac=1, random_state=50).reset_index(drop=True)

# # Seperating the data from the origional dataframe
# indexes = kept_sample.index.union(exchange_sample.index).union(refund_sample.index)

# filtered_data = data_encoded[~data_encoded.index.isin(indexes)]

# 3. Encode target separately and drop unused columns
y = balanced_data['returned']
X = balanced_data.drop(columns=['id', 'purchase_date', 'returned'])

# 4. One-hot encode features
X = pd.get_dummies(X, drop_first=False)

# 5. Train-test split (preventing leakage)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 6. Encode target AFTER the split
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# 7. Scale numerical features AFTER the split
scaler = StandardScaler()
X_train['purchase_amount'] = scaler.fit_transform(X_train[['purchase_amount']])
X_test['purchase_amount'] = scaler.transform(X_test[['purchase_amount']])


In [ ]:
# Only did to save the dataset for future use

# balanced_data = balanced_data.drop(columns=['id', 'purchase_date'])
# balanced_data['returned'] = label_encoder.transform(balanced_data['returned'])
# balanced_data['purchase_amount'] = scaler.transform(balanced_data[['purchase_amount']])
# balanced_data = pd.get_dummies(balanced_data,drop_first=False)
# balanced_data.head()

,purchase_amount,returned,purchases,return_rate,exchange_rate,kept_rate,days_since_last,cat_refund_rate,cat_exchange_rate,cat_kept_rate,channel_return_rate,channel_exchange_rate,channel_keep_rate,category_Automotive Parts & Accessories,category_Baby & Maternity Products,category_Beauty & Personal Care,category_Books & Literature,category_Clothing & Apparel,category_Electronics,category_Food & Beverages,category_Furniture & Home Decor,category_Health & Wellness Products,category_Home & Kitchen Appliances,category_Jewelry & Watches,category_Musical Instruments,category_Office Supplies & Stationery,category_Pet Supplies,category_Sports & Outdoor Equipment,category_Tools & Hardware,category_Toys & Games,product_3D Printer,product_Accent Chair,product_Accent Tables,product_Accordion,product_Acne Treatment,product_Action Camera,product_Action Figures,product_Adjustable Wrench,product_Aftershave,product_Air Fryer,product_Air Purifier,product_Aloe Vera Gel,product_Ankle Bracelet,product_Anklet,product_Anti-aging Cream,product_Arduino,product_Art Books,product_Autobiographies,product_Award-winning Books,product_Baby Bath Tub,product_Baby Books,product_Baby Bottles,product_Baby Clothes,product_Baby Food Maker,product_Baby High Chair,product_Baby Monitor,product_Baby Shoes,product_Baby Sunscreen,product_Baby Thermometer,product_Baby Toys,product_Baby Wash,product_Badge Holder,product_Bagpipes,product_Bangle,product_Basketball,product_Bass Drum,product_Bass Guitar,product_Bathroom Vanity,product_Bean Bag Chair,product_Beer,product_Belly Button Ring,product_Belt,product_Bestsellers,product_Bike Rack,product_Bird Cage,product_Birthstone Jewelry,product_Blankets,product_Blender,product_Blouse,product_Board Games,product_Boat Cover,product_Body Lotion,product_Body Oil,product_Body Scrub,product_Bodysuit,product_Bookshelf,product_Boots,product_Bracelet,product_Brake Pads,product_Bread Maker,product_Breadboard,product_Brooch Pin,product_Building Blocks,product_Cable Organizer,product_Camera,product_Camping Tent,product_Candles,product_Candy,product_Canned Goods,product_Canoe,product_Car Drying Towel,product_Car GPS Navigator,product_Car Mats,product_Car Wash Mitt,product_Car Wax,product_Card Games,product_Cardigan,product_Cargo Carrier,product_Cat Collar,product_Cat Litter,product_Cat Shampoo,product_Chain Bracelet,product_Chainsaw,product_Chandeliers,product_Changing Pad,product_Charm,product_Chips,product_Choker,product_Choker Necklace,product_Circular Saw,product_Clarinet,product_Coat,product_Coat Rack,product_Coconut Water,product_Coffee,product_Coffee Maker,product_Coffee Table,product_Comic Books,product_Computer Monitor,product_Conditioner,product_Conga Drums,product_Console Table,product_Convection Oven,product_Countertop Blender,product_Cowbell,product_Cufflinks,product_Curl Cream,product_Curtains,product_Cycling Helmet,product_Dehumidifier,product_Desk,product_Desk Calendar,product_Desk Lamp,product_Desk Organizer,product_Detox Foot Pads,product_Diaper Bag,product_Diaper Pail,product_Diaper Rash Cream,product_Didgeridoo,product_Digestive Enzymes,product_Dining Table,product_Dog Backpack,product_Dog Boots,product_Dog Collar,product_Dog Shampoo,product_Dog Sweater,product_Dress,product_Dresser,product_Drill,product_Dry Shampoo,product_Dumbbells,product_Dust Mask,product_E-book Reader,product_Earrings,product_Educational Board Games,product_Educational Books,product_Educational Toys,product_Electric Can Opener,product_Electric Guitar,product_Electric Massager,product_Electric Smoker,product_Electric Toothbrush,product_End Table,product_Energy Bars,product_Energy Drinks,product_Envelopes,product_Essays,product_Essential Oil Diffuser,product_Ethernet Cable,product_Exercise Bike,product_External Hard Drive,product_Eye Cream,product_Face Cleanser,product_Facial Mask,product_Facial Moisturizer,product_Facial Oil,product_Fantasy Novels,product_Fiber Supplements,product_Fiction Novels,product_Filing Cabinet,product_Fishing Kayak,product_Fitness

In [79]:
model_solution.dump_data(folders=['data','processed'],file_name='balanced_data.csv',obj=balanced_data)

In [ ]:
model_solution.save_model(folders=['models','data_processing_models'],file_name='standard_scaler.pkl',obj=scaler)
model_solution.save_model(folders=['models','data_processing_models'],file_name='label_encoder.pkl',obj=label_encoder)

fast api, pydantic, sqlalchemy, alembic, celery, database postgres sql

# Model Building

#### Decision Tree 

In [5]:
decision_tree = DecisionTreeClassifier(random_state=42,class_weight='balanced')
decision_tree.fit(X_train,y_train)

decision_tree_predictions = decision_tree.predict(X_test)


In [6]:
print(metrics.classification_report(y_test,decision_tree_predictions))

              precision    recall  f1-score   support

           0       0.53      0.53      0.53     12300
           1       0.53      0.52      0.53     12300
           2       0.85      0.86      0.85     12300

    accuracy                           0.64     36900
   macro avg       0.64      0.64      0.64     36900
weighted avg       0.64      0.64      0.64     36900



In [7]:
decision_tree.get_depth()

169

#### Random Forest

In [49]:
random_forest = RandomForestClassifier(random_state=42,class_weight='balanced')
random_forest.fit(X_train,y_train)
random_forest_predictions = random_forest.predict(X_test)
print(metrics.classification_report(y_test,random_forest_predictions))

              precision    recall  f1-score   support

           0       0.57      0.55      0.56     12300
           1       0.57      0.53      0.55     12300
           2       0.84      0.94      0.89     12300

    accuracy                           0.67     36900
   macro avg       0.66      0.67      0.66     36900
weighted avg       0.66      0.67      0.66     36900



#### Grid-Search Model

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2'],
    # 'max_leaf_nodes': [None, 10, 20, 50],
    'ccp_alpha': [0.0, 0.01, 0.1],
}

scoring = [
    'accuracy',
    'balanced_accuracy',
    'f1_macro',
    'f1_weighted',
    'precision_macro',
    'recall_macro',
    'neg_log_loss'
]

grid_decision_tree = DecisionTreeClassifier(random_state=42,class_weight='balanced')

grid_tree = GridSearchCV(estimator = grid_decision_tree,
                           param_grid = param_grid,
                           scoring = scoring,
                           cv = 5,
                           refit = 'f1_macro',
                           n_jobs = -1,
                           verbose = 2
                           )

grid_tree.fit(X_train,y_train)

Fitting 5 folds for each of 1296 candidates, totalling 6480 fits


GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=42),
             n_jobs=-1,
             param_grid={'ccp_alpha': [0.0, 0.01, 0.1],
                         'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [None, 5, 10, 20],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             refit='f1_macro',
             scoring=['accuracy', 'balanced_accuracy', 'f1_macro',
                      'f1_weighted', 'precision_macro', 'recall_macro',
                      'neg_log_loss'],
             verbose=2)

In [9]:
best_params = grid_decision_tree.best_estimator_.get_params()
best_tree = DecisionTreeClassifier(**best_params)
best_tree.fit(X_train,y_train)
best_tree_pred = best_tree.predict(X_test)
print(metrics.classification_report(y_test,best_tree_pred))

              precision    recall  f1-score   support

           0       0.51      0.51      0.51     12300
           1       0.51      0.51      0.51     12300
           2       0.84      0.84      0.84     12300

    accuracy                           0.62     36900
   macro avg       0.62      0.62      0.62     36900
weighted avg       0.62      0.62      0.62     36900



In [ ]:
random_forest_parameters = {
    'max_depth': [5, 10, 20],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 5, 10],
    'max_features': ["sqrt"],
    'max_samples': [0.5, 0.7],
    'n_estimators': [50, 100, 150]
}
 
score = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

grid_random_forest = RandomForestClassifier(random_state=42)

grid_forest = GridSearchCV(estimator = grid_random_forest,
                           param_grid = random_forest_parameters,
                           scoring = score,
                           cv = 5,
                           refit = 'f1_macro',
                           n_jobs = -1,
                           verbose = 2
                           )

grid_forest.fit(x_1_train, y_1_train)

Fitting 5 folds for each of 162 candidates, totalling 810 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': [5, 10, 20], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0.7],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [50, 100, 150]},
             refit='f1_macro',
             scoring=['accuracy', 'precision_macro', 'recall_macro',
                      'f1_macro'],
             verbose=2)

In [33]:
print(grid_forest.best_estimator_)

RandomForestClassifier(max_depth=20, max_samples=0.5, n_estimators=150,
                       random_state=42)


In [10]:
best_params = grid_random_forest.best_estimator_.get_params()
best_rf = RandomForestClassifier(**best_params)
best_rf.fit(X_train,y_train)

RandomForestClassifier(max_depth=20, max_samples=0.5, n_estimators=150,
                       random_state=42)

In [11]:
best_rf_predictions = best_rf.predict(X_test)
print(metrics.classification_report(y_test,best_rf_predictions))

              precision    recall  f1-score   support

           0       0.61      0.61      0.61     12300
           1       0.63      0.53      0.58     12300
           2       0.84      0.97      0.90     12300

    accuracy                           0.70     36900
   macro avg       0.69      0.70      0.69     36900
weighted avg       0.69      0.70      0.69     36900



#### `XGBOOST` Classifier

In [ ]:
xgb_params = {
              'max_depth':[6, 10, 15],
              'n_estimators':[100,200,300,400],
              'colsample_bytree':[0.6,0.8],
              'subsample':[0.6,0.8]
              }

score = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

xgb_estimator = XGBClassifier(objective='multi:softmax',num_class=3,random_state=42)

xgb_grid = GridSearchCV(
    estimator=xgb_estimator,
    param_grid=xgb_params,
    scoring=score,cv=5,
    refit='accuracy',
    n_jobs=-1,verbose=1,
    return_train_score=True
    )

xgb_grid.fit(x_1_train,y_1_train)
xgb_grid.best_estimator_

In [18]:
test_model = XGBClassifier(
    max_depth=5,
    n_estimators=200,
    colsample_bytree=0.8,
    subsample=0.8
    )
test_model.fit(X_train,y_train)
test_model_prediction = test_model.predict(X_test)
print(metrics.classification_report(y_test,test_model_prediction))

              precision    recall  f1-score   support

           0       0.60      0.58      0.59     12300
           1       0.61      0.56      0.58     12300
           2       0.85      0.95      0.90     12300

    accuracy                           0.70     36900
   macro avg       0.69      0.70      0.69     36900
weighted avg       0.69      0.70      0.69     36900

